<a href="https://colab.research.google.com/github/basse058/Big-Data-ETL/blob/main/part_one_digital_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-12 00:18:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.99MB/s    in 0.2s    

2023-03-12 00:18:28 (4.99 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz" #enter correct address here

spark.sparkContext.addFile(url)  #No need to use sparkcontext because we are reading it from a local folder

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....|2015-01-31 00:08:00|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|      

In [ ]:
# Get the number of rows in the DataFrame.
df.count()


1688884

# Transform the Data

## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(['review_id','customer_id','product_id','product_parent',to_date(df.review_date, 'yyyy-MM-dd').alias('review_date')])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-01-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-01-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-01-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-01-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-01-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-01-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-01-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-01-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-01-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-01-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-01-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-01

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(['product_id','product_title'])
products_df = products_df.dropDuplicates(['product_id'])
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000PI8FDQ|       Lost Together|
|B000PJ87FG|Til I Am Myself A...|
|B000PJC5OA|Greatest Hits Vol. 1|
|B000PJDLMA|Just Like A Vacat...|
|B000Q397XW|Lieder ohne Worte...|
|B000Q39E5S|String Quartet No...|
|B000Q3ASAS|Moments musicaux,...|
|B000Q3BCDK|Songs of Grace an...|
|B000Q3BJ8S|           Royal Sec|
|B000Q3CXNS|Oboe Concerto in ...|
|B000Q3D84G|24 Preludes, Op. ...|
|B000Q3DMQA|Toccata festiva, ...|
|B000Q3EL6A|Gloria in D Major...|
|B000Q3ESKY|Requiem, Op. 66: ...|
|B000Q3EUDY|Concerto for 2 Vi...|
|B000Q3FBUA|Dance Of The Fron...|
|B000Q3FDWQ|Knoxville: Summer...|
|B000Q3FRAE|Toccata & Fugue i...|
|B000Q3GPHI|Prayer of St. Gre...|
|B000Q3GRQW|Oboe Concerto, G ...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc
customers_df = df.groupby('customer_id').agg({"customer_id":"count"}).withColumnRenamed('count(customer_id)', 'customer_cnt')
customers_df.orderBy(desc('customer_cnt')).show()


+-----------+------------+
|customer_id|customer_cnt|
+-----------+------------+
|    7080939|        1907|
|   42418272|        1821|
|   29274627|         802|
|   53037408|         667|
|   22015347|         618|
|   35193692|         611|
|   34633160|         576|
|   14314332|         557|
|   26687122|         530|
|   34376725|         468|
|   45395259|         464|
|   12887339|         426|
|    3618115|         422|
|   29141083|         408|
|   52496677|         393|
|   13679703|         391|
|    6212523|         365|
|   52042479|         360|
|   31759399|         346|
|   50579062|         330|
+-----------+------------+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R14LVB34Z2Z53I|          5|            0|          0|   N|
|R2M0R9CWPC82LT|          5|            0|          0|   N|
| RQNQI2R3BM52A|          4|            0|          0|   N|
| RTUIX1135HUAB|          5|            0|          0|   N|
| RE3QSAT8TL010|          4|            0|          0|   N|
|R14TN65IY0NUOD|          5|            0|          0|   N|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|
|R3MJK3L7ER61TT|          5|            0|          0|   N|
|R38Z7XMCVDR0R5|          5|            1|          1|   N|
|R3FFESBQXPR7S6|          4|            0|          0|   N|
|R3QO6Z942CKH34|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://my-data-class-db.c4q3joupwllm.us-east-1.rds.amazonaws.com:5432/my-data-class-db"
config = {"user":"postgres", "password": "testpass", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)